# Fine Tune for Sex Prediction on ABIDEI Dataset
### Train/val/test from all sites - OOD test on ABIDEII

In [1]:
from dp_model.model_files.sfcn import SFCN
from dp_model import dp_loss as dpl
from dp_model import dp_utils as dpu
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import nibabel as nib
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score

## F1 and AUC_ROC

In [ ]:
print(f'F1-score: {f1_score(results_df.true_label, results_df.predicted_label)}')
print(f'ROC_AUC-score: {roc_auc_score(results_df.true_label, results_df.prob_male)}')